# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
import findspark

findspark.init("/opt/spark")

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lgr').getOrCreate()

23/05/31 17:50:40 WARN Utils: Your hostname, zack-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.40.128 instead (on interface ens33)
23/05/31 17:50:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/31 17:50:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/31 17:50:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/31 17:50:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [5]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [7]:
df.groupBy('Location').count().show()

+--------------------+-----+
|            Location|count|
+--------------------+-----+
|062 Trevor Falls ...|    1|
|066 Jenkins Walks...|    1|
|45946 Day Springs...|    1|
|143 Andrea Flat L...|    1|
|Unit 2093 Box 153...|    1|
|399 Herbert Key P...|    1|
|104 Ruben Rapid A...|    1|
|930 Carrie Harbor...|    1|
|8202 Jade Unions ...|    1|
|USCGC Bailey FPO ...|    1|
|893 Carla Trace S...|    1|
|446 Rodney Ridge ...|    1|
|30668 Isabella Fr...|    1|
|911 Kent Point An...|    1|
|078 Nunez Haven S...|    1|
|PSC 5667, Box 831...|    1|
|4972 Michael Vill...|    1|
|567 Ian Loop Lamb...|    1|
|482 Wells Mountai...|    1|
|7259 Brown Street...|    1|
+--------------------+-----+
only showing top 20 rows



In [8]:
df.groupBy('Company').count().show()

+--------------------+-----+
|             Company|count|
+--------------------+-----+
|Miller, Johnson a...|    1|
|Hunter, Reyes and...|    1|
|          Obrien PLC|    1|
|            Soto PLC|    2|
|            Todd LLC|    1|
|Smith, Marshall a...|    1|
|           Smith PLC|    1|
|          Hall Group|    1|
|Freeman, Lam and ...|    1|
|       Smith-Carroll|    1|
|Hall, Hernandez a...|    1|
|          Cannon Inc|    1|
|        White-Dennis|    1|
|Wilson, Collins a...|    1|
|Jennings, Gates a...|    1|
|     Campbell-Willis|    1|
|    Martinez-Roberts|    1|
|        Robinson PLC|    1|
|          Barton Inc|    1|
|Hernandez, Middle...|    1|
+--------------------+-----+
only showing top 20 rows



### Data Transformation


In [9]:
from pyspark.sql.functions import year, month, dayofmonth

In [10]:
df = df.withColumn("year", year(df["Onboard_date"]))
df = df.withColumn("month", month(df["Onboard_date"]))
df = df.withColumn("day", dayofmonth(df["Onboard_date"]))

In [11]:
df.head()

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1, year=2013, month=8, day=30)

In [12]:
df_col = df.select([
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'year',
 'month',
 'day',
 'Churn'])

In [13]:
clean_data = df_col.na.drop()

In [15]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=[
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'year',
 'month',
 'day'],outputCol='features')

In [16]:
data_with_features = assembler.transform(clean_data)

### Model Training

In [17]:
from pyspark.ml.classification import LogisticRegression 

In [18]:
log_reg_churn = LogisticRegression(featuresCol='features',labelCol='Churn')

In [19]:
train_data, test_data = data_with_features.randomSplit([0.7,0.3])

In [20]:
fit_model = log_reg_churn.fit(train_data)

23/05/31 17:52:09 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [21]:
results = fit_model.transform(test_data)

In [22]:
results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       1.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [39]:
model_sum = fit_model.summary

In [40]:
model_sum.predictions.describe().show()

23/05/31 20:01:53 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+-------------------+
|summary|               Age|    Total_Purchase|   Account_Manager|             Years|         Num_Sites|              year|            month|              day|             Churn|         prediction|
+-------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+-------------------+
|  count|               636|               636|               636|               636|               636|               636|              636|              636|               636|                636|
|   mean|41.948113207547166|10118.336226415084|0.4559748427672956| 5.268490566037734|  8.58805031446541|2010.7248427672955|6.193396226415095|16.36006289308176|0.1761006289308176| 0.1430817610062893|
| std

### Model Evaluation

In [23]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [24]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [25]:
AUC = my_eval.evaluate(results)

In [26]:
AUC

0.7497671169073127

In [ ]:
# Above is same as below
#pred_and_labels = fit_model.evaluate(test_data)
#auc = my_eval.evaluate(pred_and_labels.predictions)

### Predict New Data

In [27]:
new_df = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [28]:
new_df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [29]:
new_df  = new_df.withColumn("year", year(new_df["Onboard_date"]))
new_df= new_df.withColumn("month", month(new_df["Onboard_date"]))
new_df = new_df.withColumn("day", dayofmonth(new_df["Onboard_date"]))

In [30]:
new_df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'year',
 'month',
 'day']

In [31]:
new_clean_data = new_df.na.drop()

In [32]:
new_data_with_features = assembler.transform(new_clean_data)

In [33]:
predictions_for_new_data = fit_model.transform(new_data_with_features)

In [34]:
predictions_for_new_data.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

